In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date
from datetime import datetime
import re
from pandas.errors import ParserError

In [253]:
el_debate = pd.read_csv("el_debate.csv")
el_debate.drop("Unnamed: 0",axis=1,inplace=True)
el_debate.to_csv("bups/el_debate_links_bup.csv")
old_links = el_debate.url.to_list()

In [25]:
new_links = pd.DataFrame(columns=["pos","url","crawl_day"])

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(chrome_options=options)

url = "https://www.abc.es/"
driver.get(url)

page_source = driver.page_source

soup = BeautifulSoup(page_source, "lxml")

articles = soup.find_all("article")

/var/folders/6g/vz95ncc10qb5p6zp51qtwbc40000gn/T/ipykernel_25621/1048666596.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [41]:
new_links = pd.DataFrame(columns=["pos","url","crawl_day"])

for num,article in enumerate(articles):
    l = []
    for n,a in enumerate(article.findAll("a")):
        if a["href"].startswith("http"):
            href = a["href"]
        else:
            href = "{0}{1}".format(url[:-1],a["href"])         
        dominio = href.split("/")[2]
        pos = num
        l.append([dominio,pos,href,date.today()])
        df_ = pd.DataFrame(l,columns=["dominio","pos","url","crawl_day"])
        new_links = pd.concat([new_links,df_],axis=0)

new_links.drop_duplicates(inplace=True)

In [47]:
new_links

,pos,url,crawl_day,dominio
0,0,https://www.abc.es/economia/gobierno-elevara-p...,2022-10-03,www.abc.es
1,0,https://www.abc.es/economia/gobierno-elevara-p...,2022-10-03,www.abc.es
0,1,https://www.abc.es/economia/gobierno-britanico...,2022-10-03,www.abc.es
1,1,https://www.abc.es/economia/gobierno-britanico...,2022-10-03,www.abc.es
0,2,https://www.abc.es/internacional/fallaron-encu...,2022-10-03,www.abc.es
...,...,...,...,...
2,116,https://www.abc.es/loteria-del-nino/abci-como-...,2022-10-03,www.abc.es
0,117,https://horoscopo.abc.es/,2022-10-03,horoscopo.abc.es
2,117,https://horoscopo.abc.es/#disqus_thread,2022-10-03,horoscopo.abc.es
0,118,https://www.abc.es/recreo/abci-horoscopo-lunes...,2022-10-03,www.abc.es


In [48]:
x = new_links.url.iloc[2]
y = new_links.url.iloc[3]

In [49]:
print(x)
print(y)

https://www.abc.es/economia/gobierno-britanico-marcha-atras-suaviza-rebaja-fiscal-20221003092720-nt.html
https://www.abc.es/economia/gobierno-britanico-marcha-atras-suaviza-rebaja-fiscal-20221003092720-nt.html#disqus_thread


In [40]:
for col,row in new_links.iterrows():
    dominio = row["url"].split("/")[2]
    new_links.at[col,"dominio"] = dominio


['https:',
 '',
 'www.abc.es',
 'recreo',
 'abci-horoscopo-lunes-3-octubre-2022-nsv-202210022300_noticia.html#disqus_thread']

In [283]:
new_links = pd.DataFrame(columns=["pos","url","crawl_day"])

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(chrome_options=options)

url = "https://www.abc.es/"
driver.get(url)

page_source = driver.page_source

soup = BeautifulSoup(page_source, "lxml")

articles = soup.find_all("article")

for n,article in enumerate(articles):
    l = []
    for a in article.findAll("a"):
        try:
            href = "https://www.eldebate.com{0}".format(a["href"])
        except KeyError:
            pass
    pos = n

    l.append([pos,href,date.today()])
    df_ = pd.DataFrame(l,columns=["pos","url","crawl_day"])
    new_links = pd.concat([new_links,df_],axis=0)

new_links.drop_duplicates(inplace=True)
new_links.reset_index(inplace=True,drop=True)

for col,row in new_links.iterrows():
    if row["url"] in old_links:
        new_links.drop(col,inplace=True)

el_debate = pd.concat([el_debate,new_links],axis=0)
el_debate.drop_duplicates(inplace=True)
el_debate.reset_index(inplace=True,drop=True)
el_debate.to_csv("el_debate.csv")


/var/folders/6g/vz95ncc10qb5p6zp51qtwbc40000gn/T/ipykernel_6062/2703660318.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [284]:
df_full = pd.DataFrame(columns=["tags","tag","keywords","url","seccion","pos","date","title","sub","author","text","crawl_day"])

for col,row in new_links.iterrows():
    l=[]
    url = row["url"]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    head = soup.find("head")
    seccion = soup.find("meta",property="article:section")
    title = soup.find("meta",property="og:title")
    title = title["content"]
    seccion = seccion["content"]

    for i in head.find_all("meta"):
        try:
            if i["name"] == "date":
                fecha = i["content"]
            elif i["name"] == "Keywords":
                keywords = i["content"]
            elif i["name"] == "author":
                author = i["content"]
            elif i["name"] == "article:tag":
                tag = i["content"]
        
        except KeyError:
            pass

    try:
        sub = soup.find("div",class_="c-detail__subtitle").text
    except AttributeError:
        try:
            sub = soup.find("h2",class_="c-detail__subtitle").find("h2").text
        except AttributeError:
            sub = "Nan"
    
    text = []
    x = soup.find_all("div",class_="paragraph")
    for i in x:
        text.append(i.text)
    text = " ".join(text)
    
    temas = []
    x = soup.find_all("li",class_="c-detail__tags__item")
    for i in x:
        temas.append(i.text.replace("\n",""))
    
    l.append([temas,tag,keywords,row["url"],seccion,row["pos"], fecha, title, sub, author, text,date.today()])
    df_ = pd.DataFrame(l,columns=["tags","tag","keywords","url","seccion","pos","date","title","sub","author","text","crawl_day"])
    df_full = pd.concat([df_full,df_], axis=0)

df_full.url.drop_duplicates(inplace=True)

In [285]:
df_el_debate = pd.read_csv("el_debate_full.csv")
df_el_debate.drop("Unnamed: 0",axis=1,inplace=True)
df_el_debate.reset_index(inplace=True,drop=True)
df_el_debate.to_csv("bups/el_debate_full_bup.csv")

df_el_debate = pd.concat([df_el_debate,df_full],axis=0)
df_el_debate.reset_index(inplace=True,drop=True)
df_el_debate.to_csv("el_debate_full.csv")